In [ ]:
#IMPORT LIBRARIES
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from pathlib import Path
from dotenv import load_dotenv
import plotly.express as px
%matplotlib inline
import hvplot.pandas
import seaborn as sns

In [ ]:
#IMPORT DATA FOLLOWING P1_Q3 FOR THREE SECTORS/STOCKS
#Walmart, P&G, J&J. \\\
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
type(alpaca_api_key)

In [ ]:
type(alpaca_secret_key)

In [ ]:
len(alpaca_api_key)

In [ ]:
len(alpaca_secret_key)

In [ ]:
# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [ ]:
list_of_tickers = ["WMT", "JNJ", "PG","JPM", "MCD","XOM"]

# YOUR CODE HERE
# Get all Asstes
assets = api.list_assets()

In [ ]:
# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]
#tradeable

In [ ]:
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

# Display the first 10 asset tickers
asset_info_df.sample(5)

In [ ]:
tickers = ["WMT", "JNJ", "PG"]

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-88)

# Get 1 year's worth of historical data for WMT, JNJ and PG
df = api.get_barset(
    tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

df.head(10)

In [ ]:
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
# new = df.rename(columns={'time': 'Date'}) not working possibly because time is the index column; this works on other column names
#new.head(10)
df.head()

In [ ]:
#CALCULATE PERCENTAGE_CHANGE
#jnj_pct_change = df['JNJ'].pct_change().head(5)
#jnj_pct_change

In [ ]:
#pg_pct_change = df['PG'].pct_change().head(5)
#pg_pct_change

In [ ]:
#wmt_pct_change = df['WMT'].pct_change().head(5)
#wmt_pct_change

In [ ]:
daily_returns = df.pct_change()
daily_returns.head(10)

In [ ]:
#CALCULATE PORTFOLIO RETURNS / ROLLING STATSTICS TO ANALYSE PERFORMANCE



#Calculate Value of Average Daily Returns
# Use the `mean` function to calculate the mean of daily returns for tickers = ["WMT", "JNJ", "PG"]
#daily_returns = df.pct_change()


avg_daily_return_wmt = daily_returns.mean()['WMT']['close']
avg_daily_return_jnj = daily_returns.mean()['JNJ']['close']
avg_daily_return_pg = daily_returns.mean()['PG']['close']
print(f"The Mean of Daily Returns are: Walmart: {avg_daily_return_wmt}, Johnson & Johnson: {avg_daily_return_jnj}, Proctor & Gamble: {avg_daily_return_pg}")


In [ ]:
#CALCULATE VOLATITLITY
# volatitily = std
std_wmt = daily_returns.std()['WMT']['close']
std_jnj = daily_returns.std()['JNJ']['close']
std_pg = daily_returns.std()['PG']['close']
print(f"The Standard Deviation of Returns: Walmart: {std_wmt}, Johnson & Johnson: {std_jnj}, Proctor & Gamble: {std_pg}")

# first percent change

In [ ]:
# Plot rolling stats for risk (std) and return to observe any correlated patterns visually
# Plot daily portfolio return
df.plot()

In [ ]:
# Plot rolling stats for risk (std) and return to observe any correlated patterns visually
# Rolling 7-Day Mean of Closing Prices
df.rolling(window=7).mean().plot()

In [ ]:
# Rolling 7-Day Standard Deviation of Closing Prices
df.rolling(window=7).std().plot()

In [ ]:
# Rolling 30-Day Mean of Closing Prices
df.rolling(window=30).mean().plot()

In [ ]:
# Rolling 30-Day Standard Deviation of Closing Prices
df.rolling(window=30).std().plot()

In [ ]:
# Rolling 180-Day Mean of Closing Prices; adjust axis to show curve
df.rolling(window=180).mean().plot()

In [ ]:
# Rolling 180-Day Standard Deviation Closing Prices; adjust axis to show curve
df.rolling(window=180).std().plot()

In [ ]:
# Calculate the covariance of the stocks in the portfolio. # Positive covariance implies a positive relationship between all companies, 
# though not by the same fold; covariacne of any stock in the portfolio is calculated as variance of the stock return relative to the portfolio over the same
# time period
df.cov()

In [ ]:
# Calculate the Variance of the stocks in the portfolio; remember the the variaince is simply the covariance of the stock against itself; 
# Notice that the below variance is also reflected in the covariance table above
df.var()

In [ ]:
# Beta calculation
df.cov()/df.var()

In [ ]:
# Calculate the correlation coefficient of the stocks in the portfolio
df.corr()
# P&G and JNJ have the highest correlation, 0.813638
# P&G and Walmart have the least correlation, 0.192753

In [ ]:
# Use the `heatmap` function from the seaborn library to visualize correlations
price_corr = df.corr()
sns.heatmap(price_corr, vmin=-1, vmax=1)

In [ ]:
# Save the last day's closing price
jnj_last = df['JNJ'][-1:]
jnj_last

In [ ]:
# Save the last day's closing price
pg_last = df['PG'][-1:]
pg_last

In [ ]:
# Save the last day's closing price
wmt_last = df['WMT'][-1:]
wmt_last

In [ ]:
df[-1:]

In [ ]:
# Setup the Monte Carlo Parameters

In [ ]:
# Set number of simulations and trading days
num_simulations = 1000
num_trading_days = 166

# Set last closing prices of `JNJ`,WMT and PG or entire portfolio
jnj_last = df['JNJ'][-1:]
pg_last = df['PG'][-1:]
wmt_last = df['WMT'][-1:]

#Entire portfolio of stocks
#port_last = df[-1:]

# Initialize empty DataFrame to hold simulated prices for each simulation
simulated_price_df = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

#simulated_tickers_prices['symbol'] = pd.Series([asset.symbol for asset in assets])

# Run the Monte Carlo Simulation for num_simulations and num_trading_days; for x in range(number_simulations):
for n in range(num_trading_days):
    
    #simulated_port_prices = [port_last]
    simulated_jnj_prices = [jnj_last]
    simulated_pg_prices = [pg_last]
    simulated_wmt_prices = [wmt_last]
    
    
#     # Initialize the simulated prices list with the last closing price of `agg` and `spy`

#     simulated_port_prices = [agg_last_price]
#     simulated_spy_prices = [spy_last_price]
    
#      # Simulate the returns for 252 days * 30 years
    for i in range(num_trading_days):
        
        
        
#         # Calculate the simulated price using the last price within the list
        
        #simulated_port_price = simulated_port_prices[-1] * (1 + np.random.normal())
        simulated_wmt_price = simulated_wmt_prices[-1] * (1 + np.random.normal(avg_daily_return_wmt, std_wmt))
        simulated_pg_price = simulated_pg_prices[-1] * (1 + np.random.normal(avg_daily_return_pg, std_pg))
        simulated_jnj_price = simulated_jnj_prices[-1] * (1 + np.random.normal(avg_daily_return_wmt, std_wmt))

#         simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg))
#         simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_return_spy, std_dev_daily_return_spy))
        
#         # Append the simulated price to the list
        simulated_jnj_prices.append(simulated_jnj_price)
        simulated_wmt_prices.append(simulated_wmt_price)
        simulated_pg_prices.append(simulated_pg_price)
    
#     # Append a simulated prices of each simulation to DataFrame
        simulated_price_df["JNJ prices"] = pd.Series(simulated_jnj_prices)
        simulated_price_df["WMT prices"] = pd.Series(simulated_wmt_prices)
        simulated_price_df["PG prices"] = pd.Series(simulated_pg_prices)
#     simulated_price_df["AGG prices"] = pd.Series(simulated_agg_prices)
#     simulated_price_df["SPY prices"] = pd.Series(simulated_spy_prices)
    
#     # Calculate the daily returns of simulated prices
        simulated_daily_returns = simulated_price_df.pct_change()
    
#     # Set the portfolio weights (33.33%)
        weights = [0.3333, 0.3333, 0.3333]

#     # Use the `dot` function with the weights to multiply weights with each column's simulated daily returns
        portfolio_daily_returns = simulated_daily_returns.dot(weights)
    
#     # Calculate the normalized, cumulative return series
        portfolio_cumulative_returns[n] = (1 + portfolio_daily_returns.fillna(0)).cumprod()

# # Print records from the DataFrame
portfolio_cumulative_returns.head()    

In [ ]:
# Visualize the Simulation
plot_title = f"{x+1} Simulations of Cumulative Portfolio Return Trajectories Over the Next 30 Trading Years"
portfolio_cumulative_returns.plot(legend=None, title=plot_title)

In [ ]:
# Display the 90% confidence interval for the ending returns
confidence_interval = ending_cumulative_returns.quantile(q=[0.05, 0.95])
confidence_interval

In [ ]:
ending_cumulative_returns.plot(kind='hist')

In [ ]:
# Visualize the distribution of the ending returns
ending_cumulative_returns.value_counts(bins=10) / len(ending_cumulative_returns)

plt.figure();
ending_cumulative_returns.plot(kind='hist', density=True, bins=10)
plt.axvline(confidence_interval.iloc[0], color='r')
plt.axvline(confidence_interval.iloc[1], color='r')

In [ ]:
#FINAL EVALUATION: WHERE WILL THIS PORTFOLIO NET OUT?